# 🚀 Multi-Input Hybrid Deep Learning Crypto Trading System

## Complete Backend Testing Notebook

This notebook demonstrates all the core functionalities of the trading system:

1. **Price Prediction** - ML models to predict if price will go UP or DOWN
2. **Portfolio Optimization** - 4 AI strategies to allocate capital optimally
3. **Multi-Asset Analysis** - Correlation analysis and efficient frontier
4. **Model Persistence** - Save, load, and manage trained models

---

## Setup & Configuration

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# API Configuration
API_BASE_URL = "https://work-1-oqzcuohwva-ue.a.run.app/api"

def api_get(endpoint):
    """Make GET request to API"""
    response = requests.get(f"{API_BASE_URL}{endpoint}")
    return response.json()

def api_post(endpoint, data):
    """Make POST request to API"""
    response = requests.post(f"{API_BASE_URL}{endpoint}", json=data)
    return response.json()

print("✅ Setup complete!")
print(f"API URL: {API_BASE_URL}")

---
## 1. System Health Check

In [ ]:
# Check system status
print("🔍 Checking system health...\n")

# Dashboard stats
stats = api_get("/dashboard/stats")
print("📊 Dashboard Statistics:")
print(f"   - Has Advanced Model: {stats.get('has_advanced_model', False)}")
print(f"   - Total Predictions: {stats.get('total_predictions', 0)}")
print(f"   - Total Training Runs: {stats.get('total_training_runs', 0)}")

# Portfolio model info
model_info = api_get("/portfolio/model-info")
print(f"\n🧠 Portfolio Models:")
print(f"   - Deep Learning Trained: {model_info.get('deep_learning_trained', False)}")
print(f"   - RL Agent Trained: {model_info.get('rl_agent_trained', False)}")

---
## 2. Data Fetching - Multi-Exchange Support

In [ ]:
# Fetch market data for BTC/USDT
print("📈 Fetching Market Data...\n")

market_data = api_get("/market-data?symbol=BTC/USDT&timeframe=1h&limit=100")

if market_data.get('data'):
    df = pd.DataFrame(market_data['data'])
    print(f"Symbol: {market_data.get('symbol')}")
    print(f"Timeframe: {market_data.get('timeframe')}")
    print(f"Data Points: {len(df)}")
    print(f"\nLatest 5 candles:")
    display(df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].tail())
else:
    print("No data returned")

In [ ]:
# Test arbitrary date range fetching
print("📅 Testing Arbitrary Date Range Fetching...\n")

data_preview = api_post("/training/data-preview", {
    "symbol": "BTC/USDT",
    "timeframe": "1d",
    "start_date": "2022-01-01T00:00:00Z",
    "end_date": "2024-12-31T00:00:00Z"
})

print(f"Symbol: {data_preview.get('symbol')}")
print(f"Timeframe: {data_preview.get('timeframe')}")
print(f"Date Range: {data_preview.get('start_date')} to {data_preview.get('end_date')}")
print(f"Estimated Candles: {data_preview.get('estimated_candles'):,}")
print(f"Estimated Training Samples: {data_preview.get('estimated_training_samples'):,}")
print(f"Exchange: {data_preview.get('exchange')}")

---
## 3. Price Prediction - Single Asset

In [ ]:
# Make a prediction
print("🎯 Making Price Prediction...\n")

prediction = api_get("/predictions/make?symbol=BTC/USDT&use_advanced=true")

print(f"Symbol: {prediction.get('symbol')}")
print(f"Current Price: ${prediction.get('current_price', 0):,.2f}")
print(f"\n🔮 PREDICTION:")
print(f"   Direction: {prediction.get('direction_label', 'N/A')}")
print(f"   Confidence: {prediction.get('confidence', 0)*100:.1f}%")
print(f"   Model: {prediction.get('model_type', 'N/A')}")
print(f"   Timestamp: {prediction.get('timestamp', 'N/A')}")

---
## 4. Portfolio Optimization - Multi-Asset Allocation

In [ ]:
# Available assets for portfolio
print("💼 Portfolio Assets Available:\n")

assets_info = api_get("/portfolio/assets")

print(f"Total Assets: {assets_info.get('count')}")
print(f"\nAssets:")
for i, asset in enumerate(assets_info.get('assets', [])):
    print(f"   {i+1}. {asset}")

print(f"\nStrategies Available: {assets_info.get('strategies')}")
print(f"Objectives: {assets_info.get('objectives')}")
print(f"Horizons: {assets_info.get('horizons')}")

In [ ]:
# Fetch data for portfolio optimization
print("📊 Fetching Multi-Asset Data...\n")

selected_assets = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT"]

fetch_result = api_post("/portfolio/fetch-data", {
    "assets": selected_assets,
    "timeframe": "1d"
})

print(f"Status: {fetch_result.get('status')}")
print(f"Assets Fetched: {fetch_result.get('assets_fetched')}")

# Display statistics
if fetch_result.get('statistics'):
    print(f"\n📈 Asset Statistics:")
    stats_df = pd.DataFrame(fetch_result['statistics']).T
    display(stats_df[['current_price', 'expected_return', 'volatility', 'sharpe_ratio', 'data_points']])

In [ ]:
# Optimize portfolio with ALL 4 STRATEGIES
print("🎯 Optimizing Portfolio with All 4 Strategies...\n")

optimization_result = api_post("/portfolio/optimize", {
    "assets": selected_assets,
    "investment_amount": 10000,
    "strategy": "traditional_ml",
    "objective": "max_sharpe",
    "horizon": "7d",
    "compare_all": True,
    "constraints": {
        "max_weight": 30,
        "min_assets": 3
    }
})

print(f"Status: {optimization_result.get('status')}")
print(f"Recommended Strategy: {optimization_result.get('recommended')}")
print(f"\n📊 STRATEGY COMPARISON:")
print("="*60)

for strategy_name, strategy_data in optimization_result.get('strategies', {}).items():
    status = strategy_data.get('status')
    print(f"\n{strategy_name.upper()}:")
    if status == 'success':
        metrics = strategy_data.get('metrics', {})
        print(f"   Expected Return: {metrics.get('expected_return', 0)}%")
        print(f"   Volatility: {metrics.get('volatility', 0)}%")
        print(f"   Sharpe Ratio: {metrics.get('sharpe_ratio', 0)}")
        print(f"   Assets Used: {metrics.get('num_assets', 0)}")
    else:
        print(f"   Status: {status}")
        print(f"   Message: {strategy_data.get('message', 'N/A')}")

In [ ]:
# Display recommended allocation
print("💰 RECOMMENDED PORTFOLIO ALLOCATION ($10,000):\n")

recommended = optimization_result.get('recommended', 'traditional_ml')
allocations = optimization_result.get('strategies', {}).get(recommended, {}).get('allocations', [])

if allocations:
    alloc_df = pd.DataFrame(allocations)
    alloc_df['amount_display'] = alloc_df['amount'].apply(lambda x: f"${x:,.2f}")
    alloc_df['weight_display'] = alloc_df['weight'].apply(lambda x: f"{x}%")
    
    print(f"Strategy: {recommended}")
    print("\nAllocation:")
    display(alloc_df[['symbol', 'weight_display', 'amount_display', 'expected_return']])
    
    print(f"\nTotal Investment: ${alloc_df['amount'].sum():,.2f}")

---
## 5. Correlation Analysis

In [ ]:
# Get correlation matrix
print("🔗 Asset Correlation Analysis...\n")

correlation = api_get("/portfolio/correlation")

if correlation.get('status') == 'success':
    corr_data = correlation.get('data', {})
    assets = corr_data.get('assets', [])
    matrix = corr_data.get('matrix', [])
    
    corr_df = pd.DataFrame(matrix, index=assets, columns=assets)
    print("Correlation Matrix:")
    display(corr_df.style.background_gradient(cmap='RdYlGn', vmin=-1, vmax=1))
    
    # Diversification pairs
    div_pairs = correlation.get('diversification_pairs', [])
    if div_pairs:
        print(f"\n🎯 Best Diversification Pairs (correlation < 0.3):")
        for pair in div_pairs[:5]:
            print(f"   {pair[0]} ↔ {pair[1]}: {pair[2]}")

---
## 6. Efficient Frontier

In [ ]:
# Get efficient frontier
print("📈 Efficient Frontier Analysis...\n")

frontier = api_get(f"/portfolio/efficient-frontier?assets={','.join(selected_assets)}")

if frontier.get('status') == 'success':
    frontier_data = frontier.get('frontier', [])
    
    if frontier_data:
        frontier_df = pd.DataFrame(frontier_data)
        print(f"Frontier Points: {len(frontier_df)}")
        print(f"\nRisk-Return Tradeoff:")
        display(frontier_df.head(10))
        
        # Best Sharpe point
        best_sharpe = frontier_df.loc[frontier_df['sharpe'].idxmax()]
        print(f"\n⭐ Optimal Point (Max Sharpe):")
        print(f"   Return: {best_sharpe['return']}%")
        print(f"   Risk: {best_sharpe['volatility']}%")
        print(f"   Sharpe: {best_sharpe['sharpe']}")

---
## 7. Model Training

In [ ]:
# Check training status
print("🏋️ Model Training Status...\n")

training_status = api_get("/training/advanced/status")

print(f"Is Training: {training_status.get('is_training', False)}")
print(f"Current Model: {training_status.get('current_model', 'N/A')}")

if training_status.get('final_accuracy'):
    print(f"\nLast Training Results:")
    print(f"   Accuracy: {training_status.get('final_accuracy', 0)*100:.2f}%")
    print(f"   Total Epochs: {training_status.get('total_epochs', 0)}")
    
if training_status.get('data_info'):
    data_info = training_status['data_info']
    print(f"\nTraining Data:")
    print(f"   Samples: {data_info.get('samples', 0):,}")
    print(f"   Features: {data_info.get('features', 0)}")

In [ ]:
# List available network architectures
print("🧠 Available Network Architectures:\n")

architectures = [
    ("LSTM", "Long Short-Term Memory - Good for sequential patterns"),
    ("GRU", "Gated Recurrent Unit - Faster training"),
    ("Bi-LSTM", "Bidirectional LSTM - Captures both directions"),
    ("Transformer", "Attention-based - Best for long sequences"),
    ("CNN-LSTM", "Convolutional + LSTM - Feature extraction"),
    ("Attention GRU", "GRU with attention mechanism"),
    ("TCN", "Temporal Convolutional Network"),
    ("RL DQN", "Deep Q-Network - Reinforcement Learning"),
    ("RL PPO", "Proximal Policy Optimization"),
    ("TCN-GNN-LSTM", "Hybrid: TCN + Graph Neural Network + LSTM"),
    ("Multi-Model Ensemble", "Combines multiple models")
]

for i, (name, desc) in enumerate(architectures, 1):
    print(f"{i:2}. {name:20} - {desc}")

---
## 8. Model Persistence - Save/Load/Delete

In [ ]:
# List all saved portfolio models
print("💾 Saved Portfolio Models...\n")

saved_models = api_get("/portfolio/models/list")

print(f"Total Saved Models: {saved_models.get('total_models', 0)}")

print(f"\n🧠 Deep Learning Models ({len(saved_models.get('deep_learning_models', []))})")
for model in saved_models.get('deep_learning_models', []):
    print(f"   • {model['model_name']}")
    print(f"     Assets: {model.get('n_assets', 0)} | Params: {model.get('model_params', 0):,}")
    print(f"     Created: {model.get('created_at', 'N/A')}")

print(f"\n🤖 RL Agent Models ({len(saved_models.get('rl_agent_models', []))})")
for model in saved_models.get('rl_agent_models', []):
    print(f"   • {model['model_name']}")
    print(f"     Assets: {model.get('n_assets', 0)}")
    if model.get('training_result'):
        tr = model['training_result']
        print(f"     Training Return: {tr.get('total_return', 0):.2f}%")

In [ ]:
# Example: Load a saved model
print("📂 Loading a Saved Model...\n")

dl_models = saved_models.get('deep_learning_models', [])
if dl_models:
    model_to_load = dl_models[0]  # Load the first one
    print(f"Loading: {model_to_load['model_name']}")
    
    load_result = api_post("/portfolio/models/load", {
        "model_type": "deep_learning",
        "model_path": model_to_load['model_path']
    })
    
    print(f"Status: {load_result.get('status')}")
    print(f"Assets: {load_result.get('n_assets', 0)}")
    print(f"Asset Names: {load_result.get('asset_names', [])}")
else:
    print("No saved DL models to load")

---
## 9. AI Advisor (LLM Integration)

In [ ]:
# Test AI Advisor
print("🤖 AI Advisor Test...\n")

advisor_response = api_post("/llm/chat", {
    "message": "What is the current market sentiment for Bitcoin and should I invest now?",
    "model": "gpt-4o"
})

if advisor_response.get('response'):
    print(f"Model: {advisor_response.get('model', 'N/A')}")
    print(f"\n📝 AI Response:")
    print("-" * 50)
    print(advisor_response['response'][:1000])  # First 1000 chars
else:
    print(f"Error: {advisor_response.get('error', 'Unknown error')}")

---
## 10. Complete Workflow Demo

In [ ]:
def complete_portfolio_workflow(investment_amount=10000, assets=None):
    """
    Complete workflow: Fetch Data → Analyze → Optimize → Recommend
    """
    if assets is None:
        assets = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT"]
    
    print("="*60)
    print("🚀 COMPLETE PORTFOLIO OPTIMIZATION WORKFLOW")
    print("="*60)
    print(f"\n💰 Investment Amount: ${investment_amount:,}")
    print(f"📊 Selected Assets: {len(assets)}")
    
    # Step 1: Fetch data
    print("\n[1/4] Fetching market data...")
    fetch = api_post("/portfolio/fetch-data", {"assets": assets, "timeframe": "1d"})
    print(f"      ✅ Fetched data for {fetch.get('assets_fetched', 0)} assets")
    
    # Step 2: Analyze correlations
    print("\n[2/4] Analyzing correlations...")
    corr = api_get("/portfolio/correlation")
    if corr.get('diversification_pairs'):
        best_pair = corr['diversification_pairs'][0]
        print(f"      ✅ Best diversification: {best_pair[0]} ↔ {best_pair[1]} (corr: {best_pair[2]})")
    
    # Step 3: Optimize
    print("\n[3/4] Running optimization...")
    opt = api_post("/portfolio/optimize", {
        "assets": assets,
        "investment_amount": investment_amount,
        "objective": "max_sharpe",
        "compare_all": True,
        "constraints": {"max_weight": 30, "min_assets": 3}
    })
    print(f"      ✅ Recommended strategy: {opt.get('recommended', 'N/A')}")
    
    # Step 4: Display results
    print("\n[4/4] Final Recommendations:")
    print("="*60)
    
    recommended = opt.get('recommended', 'traditional_ml')
    strategy_data = opt.get('strategies', {}).get(recommended, {})
    
    if strategy_data.get('allocations'):
        print(f"\n💼 OPTIMAL ALLOCATION (${investment_amount:,}):")
        print("-"*40)
        for alloc in strategy_data['allocations']:
            bar = "█" * int(alloc['weight'] / 2)
            print(f"   {alloc['symbol']:12} {bar:15} {alloc['weight']:5.1f}%  ${alloc['amount']:>8,.2f}")
        
        metrics = strategy_data.get('metrics', {})
        print(f"\n📊 EXPECTED PERFORMANCE:")
        print(f"   Expected Return: {metrics.get('expected_return', 0):+.2f}%")
        print(f"   Risk (Volatility): {metrics.get('volatility', 0):.2f}%")
        print(f"   Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
    
    return opt

# Run the complete workflow
result = complete_portfolio_workflow(investment_amount=10000)

---
## Summary

### System Capabilities:

1. **Multi-Exchange Data Fetching** - Binance, OKX, and more
2. **Arbitrary Date Range** - Fetch historical data from 2017 to today
3. **Price Prediction** - 11+ neural network architectures
4. **Portfolio Optimization** - 4 AI strategies:
   - Traditional + ML (Mean-Variance with ML predictions)
   - Deep Learning (LSTM + Attention)
   - RL Agent (PPO)
   - Hybrid Ensemble
5. **Correlation Analysis** - Find diversification opportunities
6. **Efficient Frontier** - Optimal risk-return tradeoff
7. **Model Persistence** - Save, load, delete trained models
8. **AI Advisor** - LLM-powered market analysis

### API Endpoints:

| Endpoint | Description |
|----------|-------------|
| `/api/market-data` | Fetch OHLCV data |
| `/api/predictions/make` | Get price prediction |
| `/api/portfolio/fetch-data` | Fetch multi-asset data |
| `/api/portfolio/optimize` | Get optimal allocation |
| `/api/portfolio/correlation` | Correlation matrix |
| `/api/portfolio/models/list` | List saved models |
| `/api/portfolio/models/save` | Save trained model |
| `/api/portfolio/models/load` | Load saved model |
| `/api/llm/chat` | AI Advisor chat |